<a href="https://colab.research.google.com/github/smwongela/Android_Test/blob/master/5_GetEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
root = "/content/drive/MyDrive/Jacaranda/project_nlp_psu-master"

In [4]:
#!unzip {root}/checkpoint/xlm-roberta-large-2/checkpoint.zip -d {root}/checkpoint/xlm-roberta-large-2/

In [5]:
#!mv {root}/checkpoint/xlm-roberta-large-2/checkpoint/xlm-roberta-large-2/* {root}/checkpoint/xlm-roberta-large-2/

In [6]:
!pip install sentencepiece transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 24.9 MB/s 
     |████████████████████████████████| 4.9 MB 58.8 MB/s 
     |████████████████████████████████| 6.6 MB 62.3 MB/s 
     |████████████████████████████████| 120 kB 73.0 MB/s 


In [7]:
import os
import pandas as pd
import numpy as np
import torch

os.chdir(root)

In [8]:
from model_file.pred_single import My_XLMRobertaModel
from model_file.pred_single import preprocess

from typing import List, Optional, Tuple, Union

from transformers import XLMRobertaConfig, SequenceClassifierOutput


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

ImportError: ignored

In [ ]:
class XMLRAnalayst(My_XLMRobertaModel):
  
  def __init__(self,pretrain_model_path,
	                  path_saved_model,
					  learning_rate,
					  n_class,
					  path_table,
					  epochs,
					  warm_step,
					  train_batch_size,
					  test_batch_size,
					  max_len,
					  gpu):

    super().__init__(pretrain_model_path,
	                  path_saved_model,
					  learning_rate,
					  n_class,
					  path_table,
					  epochs,
					  warm_step,
					  train_batch_size,
					  test_batch_size,
					  max_len,
            gpu)

            
    self.model.eval()  
    pretrain_model_name = "xlm-roberta-large"
    self.config = XLMRobertaConfig.from_pretrained(pretrain_model_name) # obtain Roberta model config
    self.config.num_labels = n_class
    #self.roberta = RobertaModel(self.config, add_pooling_layer=False)
    
  def eval_classifier(self, features):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.model.classifier.dropout(x)
        x = self.model.classifier.dense(x)
        x = torch.tanh(x)
        x_emb = self.model.classifier.dropout(x)
        x = self.model.classifier.out_proj(x_emb)

        return {'emb': x_emb, 'proj': x}

  def forward(  self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None):
    
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.model.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        
        classifier_out = self.eval_classifier(sequence_output)

        classifier_embeddings = classifier_out['emb']
        logits = classifier_out['proj']

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return outputs, classifier_embeddings
      
      






In [ ]:
path_project_folder=root
# set the dataset path
path_table = os.path.join(path_project_folder, "intent_label_relation.csv")

# set trained model path
save_model_path = os.path.join(path_project_folder, "checkpoint")
path_saved = os.path.join(save_model_path, "xlm-roberta-large")
path_best_model = os.path.join(save_model_path, 'xlm-roberta-large-2')

my_xml_roberta_model = XMLRAnalayst(pretrain_model_path = path_best_model,
		                                    path_saved_model = path_saved,
		                                    learning_rate = 2e-5,
		                                    n_class = 58,
											path_table=path_table,
		                                    epochs = 3,
											warm_step=100,
		                                    train_batch_size = 8,
		                                    test_batch_size = 128,
		                                    max_len = 250,
		                                    gpu = True
											)
#my_xml_roberta_model.model.eval()

processor = preprocess(tokenizer_vocab_path = path_best_model,
							tokenizer_max_len = 250
							)

In [ ]:
my_xml_roberta_model.model.classifier

In [ ]:
test_df = pd.read_csv(root+"/split_test_data.csv")
test_df.head()

In [ ]:
test_df.shape

## Getting the XLM-R embeddings and classifier embedding

In [ ]:
text = test_df.text.iloc[2]
input_ids, attention_mask = processor.encode_fn(text)
(input_ids, attention_mask) = (input_ids.to("cuda:0"), attention_mask.to("cuda:0"))

with torch.no_grad():
  xmlr_out, classifier_emb = my_xml_roberta_model.forward(input_ids, attention_mask)
xmlr_out, classifier_emb = xmlr_out.last_hidden_state.detach().cpu().numpy(), classifier_emb.detach().cpu().numpy()
#logits.shape

In [ ]:
xmlr_out.shape

In [ ]:
classifier_emb.shape

In [ ]:
text = list(test_df.text.iloc[0:5])
text_processed, attention_ids = processor.encode_fn(text)
input_ids, attention_mask = processor.encode_fn(text)

with torch.no_grad():
  logits = my_xml_roberta_model.predict_single(input_ids, attention_mask))
logits.shape

# Final Scripts
Getting the logits and the [XLMRoberta] embeddings for all the test set and storing them for further analyzes. 

In [ ]:
class EmbedLogitClass:
  def __init__(self,data):
    self.data = data
  def __call__(self):
    return self.data

In [ ]:
logit = EmbedLogitClass(input_embed)
logit

In [ ]:
def getEmbedLogits(text_list):
  input_ids, attention_mask = processor.encode_fn(text_list)
  (input_ids, attention_mask) = (input_ids.to("cuda:0"), attention_mask.to("cuda:0"))

  with torch.no_grad():
    xmlr_out, classifier_emb = my_xml_roberta_model.forward(input_ids, attention_mask)
  xmlr_out, classifier_emb = xmlr_out.last_hidden_state.detach().cpu().numpy(), classifier_emb.detach().cpu().numpy()

  return xmlr_out, classifier_emb

In [ ]:
%%time

batch_size = 1
n = test_df.shape[0]
split_points = list(range(0, n, batch_size)) + [n]

for step in range(len(split_points)-1):
  low = split_points[step]
  high = split_points[step+1]

  text_list = test_df.text.iloc[low:high].to_list()
  xmlr_out, classifier_emb = getEmbedLogits(text_list)

  break
  

In [ ]:
test_df["xmlr_out"] = np.nan
test_df["classifier_emb"] = np.nan

test_df.loc[low:(high-1), ["logits", "embed"]] = input_logits, input_embed 

In [ ]:
test_df.to_pickle(root+"/test_df_logits_embed.pkl")